In [3]:
!pip install selenium


[notice] A new release of pip is available: 23.1.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [110]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

driver = webdriver.Chrome()

# def get_url():
#     template=  'https://www.amazon.in/s?k=bags'
#     search_term = search_term.replace(' ', '+')
    
#     url = template.format()
    
#     url += '&page{}'
    
    
#     return url

def extract_record(item):
    atag = item.h2.a
    name = atag.text.strip()
    url = 'https:www.amaxon.in' + atag.get('href')
    
    try:
        price_parent = item.find('span', 'a-price')
        price=price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
        
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (name, price, rating, review_count, url)
    
    
    return result

def main():
    driver = webdriver.Chrome()
    
    records = []
    url = 'https://www.amazon.in/s?k=bags&page={}'
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type':'s-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
            
    driver.close()
    
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Name', 'Price', 'Rating', 'Review_count', 'Url'])
        writer.writerows(records)


In [111]:
main()